#  US Gap Analysis Project - WV Breeding Bird Atlas Data Comparison 
Nathan Tarr and Jessie Jordan

## Species occurring in West Virginia
We investigated the agreement between WV Breeding Bird Atlas (2011-2015) and USGS Gap Analysis Project data on the species occupying West Virginia.

We combined a list of species that GAP predicted to be in WV during the breeding season with the WVBBA data by linking records on common names.  We resolved unmatched names, although very few existed.

# What's with CAWA missing?  Others?

In [1]:
import pandas as pd
import repo_functions as fun 
import numpy as np
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 15)

# Read in the spp crosswalk
crosswalk = pd.read_csv(fun.dataDir + "SpeciesLists/WV_GAP_Atlas.csv", header=0, names=["GAP_sci_name", "GAP_code", 
                                                                                        "common_name", "GAP_habitat(ha)",
                                                                                        "WVBBA_sci_name", "WVBBA_individuals",
                                                                                        "WVBBA_points", "WVBBA_rate"])
print("The first 5 records")
print(crosswalk.head().T)

The first 5 records
                                  0                    1                   2                   3                    4
GAP_sci_name       Falco sparverius  Thryomanes bewickii  Passerina caerulea  Quiscalus quiscula   Geothlypis trichas
GAP_code                     bamkex               bbewrx              bblgrx              bcogrx               bcoyex
common_name        American Kestrel        Bewick's Wren       Blue Grosbeak      Common Grackle  Common Yellowthroat
GAP_habitat(ha)         1.91945e+06              1454.67         1.66457e+06         2.09797e+06          4.31626e+06
WVBBA_sci_name                  NaN                  NaN                 NaN  Quiscalus quiscula   Geothlypis trichas
WVBBA_individuals               NaN                  NaN                 NaN                 841                  734
WVBBA_points                    NaN                  NaN                 NaN                 274                  545
WVBBA_rate                      NaN 

### eBird
We pulled a list of species, and the frequency of detection from the eBird API via the R rebird package.  The table returned had the following columns for WV during the months of May and June and years 2011 to 2015:

"monthQt": month and week (eBird data divides each month by four
weeks)

"comName": species common name

"frequency": proportion of times the species was seen in a
specified week

"sampleSize" number of complete eBird checklists submitted for
specified given week.

We aggregated weekly mean frequencies of detection to a single value by calculating their mean.

In [11]:
# Ebird pulled via rebird frequency 2011 to 2015 in WV
eBird =(pd.read_csv(fun.dataDir + "SpeciesLists/ebird_WV_2011_2015.csv")
        [lambda x: x['frequency'] > 0]
        [lambda x: x['comName'].str.contains(' sp.') == False]
        [lambda x: x['monthQt'].str.contains('May|June')]
        [lambda x: x['comName'].str.contains('hybrid') == False]
        [lambda x: x['comName'].str.contains('/') == False]
        .drop(['monthQt'], axis=1)
        .groupby('comName').mean()
        .drop(['sampleSize'], axis=1)
        .rename(columns={'frequency': 'eBird_mean_freq'})
        .reset_index()
        .sort_values(by=['eBird_mean_freq'], ascending=False))
eBird.loc[eBird[eBird['comName'] == 'Black-crowned Night-Heron'].index, 'comName'] = 'Black-crowned Night-heron'
eBird.loc[eBird[eBird['comName'] == 'Eastern Whip-poor-will'].index, 'comName'] = 'Whip-poor-will'
eBird.loc[eBird[eBird['comName'] == 'Eurasian Collared-Dove'].index, 'comName'] = 'Eurasian Collared-dove'
eBird['COMMON_NAME'] = [x.upper() for x in eBird['comName']]

We merged the GAP-WVBBA species list crosswalk with the eBird species list to facilitate comparisons of species lists.

In [12]:
# Merge the eBird, WV, and GAP lists
crosswalk["COMMON"] = [x.upper() for x in crosswalk['common_name']]
df = (pd.merge(crosswalk, eBird, left_on='COMMON', right_on='COMMON_NAME',how='outer')
      .drop(["COMMON", "COMMON_NAME"], axis=1)
      .sort_values(by=['common_name']))
df.to_csv(fun.dataDir + "/SpeciesLists/merged_spp_lists.csv")

### Which species were on the eBird and GAP list but not detected by WVBBA?

In [13]:
a = (df[(df['WVBBA_individuals'].isnull() == True) & 
        (df['GAP_code'].isnull() == False) &
        (df['comName'].isnull() == False)]
     .drop(["WVBBA_sci_name", "WVBBA_individuals", "WVBBA_points", "WVBBA_rate", "comName"], axis=1)
     .sort_values("GAP_habitat(ha)", ascending=False)
     .set_index(["common_name"]))
print(str(len(a)) + " species")
print(a)

71 species
                                         GAP_sci_name GAP_code  GAP_habitat(ha)  eBird_mean_freq
common_name                                                                                     
Common Nighthawk                     Chordeiles minor   bconix       6264655.47         0.007216
Sharp-shinned Hawk                 Accipiter striatus   bsshax       6165425.52         0.012097
Great Horned Owl                     Bubo virginianus   bghowx       6020121.24         0.002978
Eastern Screech-owl                    Megascops asio   beasox       5659328.88         0.005242
White-throated Sparrow         Zonotrichia albicollis   bwtspx       5423426.46         0.053249
Pine Siskin                           Carduelis pinus   bpisix       5397580.08         0.016178
Ruffed Grouse                         Bonasa umbellus   brugrx       5245966.08         0.025642
Cooper's Hawk                      Accipiter cooperii   bcohax       5158733.49         0.024703
Brown Creeper      

### Which species were submitted on the eBird list but not GAP or WVBBA?

In [14]:
b = (df[(df['WVBBA_individuals'].isnull() == True) & 
     (df['GAP_code'].isnull() == True) &
     (df['comName'].isnull() == False)]
     .sort_values("eBird_mean_freq", ascending=False)
     .set_index(["comName"]))
print(b[["eBird_mean_freq"]])

                               eBird_mean_freq
comName                                       
Spotted Sandpiper                     0.071560
Blackpoll Warbler                     0.046135
Solitary Sandpiper                    0.041571
Tennessee Warbler                     0.034274
Osprey                                0.027728
Least Sandpiper                       0.019457
Palm Warbler                          0.016924
Cape May Warbler                      0.013832
Bay-breasted Warbler                  0.013348
Common Merganser                      0.012187
Lesser Yellowlegs                     0.010913
Greater Yellowlegs                    0.008131
Semipalmated Plover                   0.007858
Ring-billed Gull                      0.006531
Wilson's Warbler                      0.006330
Great Egret                           0.005271
Semipalmated Sandpiper                0.005211
Lincoln's Sparrow                     0.005165
Mallard (Domestic type)               0.004767
Gray-cheeked 

### Were any species detected by WVBBA but not submitted to eBird or predicted present by GAP?

In [15]:
c = (df[(df['WVBBA_individuals'].isnull() == False) & 
     (df['GAP_code'].isnull() == True) &
     (df['comName'].isnull() == True)])
if len(c) > 0:
    print(c)
else:
    print("No")

No


### Were any species predicted present by GAP but not detected by WVBBA or submitted to eBird?

In [16]:
d = (df[(df['WVBBA_individuals'].isnull() == True) & 
     (df['GAP_code'].isnull() == False) &
     (df['comName'].isnull() == True)]
    .set_index(["common_name"])
    .sort_values(["GAP_habitat(ha)"], ascending=False))
if len(d) > 0:
    print(d[["GAP_code", "GAP_sci_name", "GAP_habitat(ha)"]])
else:
    print("No")

                       GAP_code           GAP_sci_name  GAP_habitat(ha)
common_name                                                            
White-winged Crossbill   bwwcrx       Loxia leucoptera       1322059.14
American Tree Sparrow    batspx       Spizella arborea       1179375.39
Lapland Longspur         blalox   Calcarius lapponicus       1076582.16
Short-eared Owl          bseowx          Asio flammeus       1049761.26
Rough-legged Hawk        brlhax          Buteo lagopus       1007153.37
Hoary Redpoll            bhorex   Carduelis hornemanni        700331.76
Northern Shrike          bnshrx       Lanius excubitor        427751.10
Snowy Owl                bsnowx        Bubo scandiacus        156197.88
Lark Sparrow             blaspx   Chondestes grammacus         34328.16
Pine Grosbeak            bpigrx    Pinicola enucleator         19101.96
Eurasian Collared-dove   beucdx  Streptopelia decaocto          8702.73
Snow Bunting             bsnbux  Plectrophenax nivalis          

### Which species were predicted present by GAP but not detected by WVBBA? eBird ignored here.

In [17]:
e = (df[(df['WVBBA_individuals'].isnull() == True) & 
     (df['GAP_code'].isnull() == False)]
    .set_index(["common_name"])
    .sort_values(["common_name"], ascending=True))
if len(e) > 0:
    print(e[["GAP_code", "GAP_sci_name", "GAP_habitat(ha)"]])
else:
    print("No")

                          GAP_code                GAP_sci_name  GAP_habitat(ha)
common_name                                                                    
American Bittern            bambix       Botaurus lentiginosus         92506.14
American Black Duck         babdux               Anas rubripes        144851.85
American Coot               bamcox            Fulica americana          4085.55
American Kestrel            bamkex            Falco sparverius       1919452.23
American Tree Sparrow       batspx            Spizella arborea       1179375.39
American Woodcock           bamwox              Scolopax minor       2828272.41
Bald Eagle                  bbaeax    Haliaeetus leucocephalus         19142.82
Bank Swallow                bbansx             Riparia riparia        400276.44
Barn Owl                    bbanox                   Tyto alba        141250.86
Bewick's Wren               bbewrx         Thryomanes bewickii          1454.67
Black Vulture               bblvux      